In [3]:
import random
import plotly.graph_objects as go
#random.seed(99)
import pandas as pd
import numpy as np
import scipy
import dask.dataframe as dd
import seaborn as sns
import dask.datasets
from dask_sql import Context
import datetime
import time
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

pd.options.mode.chained_assignment = None

In [79]:
def count_users_requests(name, df20_raw,df21_raw,df22_raw,df23_raw):
    df_all = dd.concat([df20_raw,df21_raw,df22_raw,df23_raw])
    users_itogo = df_all.groupby('rubricName').user.nunique().compute()
    users_itogo_result = df_all.user.nunique().compute()

    df20_result_cnt = df20_raw.user.nunique().compute()
    df21_result_cnt = df21_raw.user.nunique().compute()
    df22_result_cnt = df22_raw.user.nunique().compute()
    df23_result_cnt = df23_raw.user.nunique().compute()

    print(users_itogo_result, 
    df20_result_cnt,
    df21_result_cnt,
    df22_result_cnt,
    df23_result_cnt,)

    users_itogo.to_excel('{}users_itogo.xlsx'.format(name))

def agg_df(df, year):
    start_dt = datetime.datetime.now()
    print('Start:', start_dt)
    df1 = df[['user', 'rubricName', 'date']]
    
    try:
        pd_df1_req_cnt = pd.read_excel(f'df1_req_cnt_{year}.xlsx')
        pd_df1_user_cnt = pd.read_excel(f'df1_user_cnt_{year}.xlsx')
        df1_agg = pd_df1_req_cnt.merge(pd_df1_user_cnt, left_index=True, right_index=True)
    except:
        df1_user_cnt = df1.groupby('rubricName').user.nunique().compute()
        df1_req_cnt = df1.groupby('rubricName').date.count().compute()
        df1_req_cnt.to_excel(f'df1_req_cnt_{year}.xlsx')
        df1_user_cnt.to_excel(f'df1_user_cnt_{year}.xlsx')
        pd_df1_req_cnt = pd.DataFrame(df1_req_cnt)
        pd_df1_user_cnt = pd.DataFrame(df1_user_cnt)
        df1_agg = pd_df1_req_cnt.merge(pd_df1_user_cnt, left_on='rubricName', right_on='rubricName')
    
    df1_agg = df1_agg.sort_values('date', ascending=False)
    df1_agg['rubricName'] = df1_agg['rubricName_x']
    df1_agg = df1_agg.set_index('rubricName')[['date', 'user']].fillna(0).astype(int)
    df1_agg.to_excel(f'df1_agg_{year}.xlsx', index=False)
    end_dt = datetime.datetime.now()
    print('End:', end_dt)
    print('Duration:', end_dt-start_dt)
    return df1_agg

def row_number(df, col, sort_col, desc=True, filter_df=True, index=45, filter_zero=False):
    if col not in df.columns:
        df[col] = ''
    if filter_df:
        rows_to_count = df[df[col]].sort_values(sort_col,ascending=not desc).loc[:,[col]].index[:index]
    elif filter_zero:
        rows_to_count = df[df[col]!=0].sort_values(sort_col,ascending=not desc).loc[:,[col]].index[:index]
    else:
        rows_to_count = df.sort_values(sort_col,ascending=not desc).loc[:,[col]].index[:index]
        df_shape = df.shape
    rows_to_zero = [i for i in df.index if i not in rows_to_count] 
    n = 0
    for r0 in rows_to_zero:
        df.loc[r0, col] = 0
    for r in rows_to_count:
        n += 1
        df.loc[r, col] = n
    
    return df

def count_ratings(df):
    df_user_max = df.loc[:,['Пользователи 2020','Пользователи 2021','Пользователи 2022','Пользователи 2023']].max(axis=1)
    df_user_min = df.loc[:,['Пользователи 2020','Пользователи 2021','Пользователи 2022','Пользователи 2023']].min(axis=1)

    df['Рейтинг по пользователям 2020'] = df_user_max == df.loc[:,['Пользователи 2020']].iloc[:,0]
    df['Рейтинг по пользователям 2021'] = df_user_max == df.loc[:,['Пользователи 2021']].iloc[:,0]
    df['Рейтинг по пользователям 2022'] = df_user_max == df.loc[:,['Пользователи 2022']].iloc[:,0]
    df['Рейтинг по пользователям 2023'] = df_user_max == df.loc[:,['Пользователи 2023']].iloc[:,0]

    df['Антирейтинг по пользователям 2020'] = df_user_min == df.loc[:,['Пользователи 2020']].iloc[:,0]
    df['Антирейтинг по пользователям 2021'] = df_user_min == df.loc[:,['Пользователи 2021']].iloc[:,0]
    df['Антирейтинг по пользователям 2022'] = df_user_min == df.loc[:,['Пользователи 2022']].iloc[:,0]
    df['Антирейтинг по пользователям 2023'] = df_user_min == df.loc[:,['Пользователи 2023']].iloc[:,0]

    df['Рейтинг постоянного\nроста пользователей'] = (df['Пользователи 2020'] < df['Пользователи 2021']
        & (df['Пользователи 2021'] < df['Пользователи 2022'])
        & (df['Пользователи 2022'] < df['Пользователи 2023']))

    df['Рейтинг постоянного\nпадения пользователей'] = ((df['Пользователи 2020'] > df['Пользователи 2021']) 
        & (df['Пользователи 2021'] > df['Пользователи 2022'])
        & (df['Пользователи 2022'] > df['Пользователи 2023']))

    df['Рейтинг по пользователям ВНВН'] = ((df['Пользователи 2020'] > df['Пользователи 2021']) 
        & (df['Пользователи 2021'] < df['Пользователи 2022'])
        & (df['Пользователи 2022'] > df['Пользователи 2023']))

    df['Рейтинг по пользователям НВНВ'] = ((df['Пользователи 2020'] < df['Пользователи 2021']) 
        & (df['Пользователи 2021'] > df['Пользователи 2022'])
        & (df['Пользователи 2022'] < df['Пользователи 2023']))

    df['Рейтинг стабильности\nчисла пользователей'] = abs(df_user_min / df_user_max - 1)

    df = row_number(df, 'Рейтинг по пользователям 2020', 'Пользователи 2020')
    df = row_number(df, 'Рейтинг по пользователям 2021', 'Пользователи 2021')
    df = row_number(df, 'Рейтинг по пользователям 2022', 'Пользователи 2022')
    df = row_number(df, 'Рейтинг по пользователям 2023', 'Пользователи 2023')
    df = row_number(df, 'Рейтинг по пользователям, итого', 'Пользователи, итого', filter_df=False)

    df = row_number(df, 'Антирейтинг по пользователям 2020', 'Пользователи 2020', desc=False)
    df = row_number(df, 'Антирейтинг по пользователям 2021', 'Пользователи 2021', desc=False)
    df = row_number(df, 'Антирейтинг по пользователям 2022', 'Пользователи 2022', desc=False)
    df = row_number(df, 'Антирейтинг по пользователям 2023', 'Пользователи 2023', desc=False)

    df = row_number(df, 'Рейтинг постоянного\nроста пользователей', 'Пользователи, итого')
    df = row_number(df, 'Рейтинг постоянного\nпадения пользователей', 'Пользователи, итого')
    df = row_number(df, 'Рейтинг по пользователям ВНВН', 'Пользователи 2023')
    df = row_number(df, 'Рейтинг по пользователям НВНВ', 'Пользователи 2023')
    df = row_number(df, 'Рейтинг стабильности\nчисла пользователей', 'Пользователи, итого', filter_df=False)

    df_req_max = df.loc[:,['Запросы 2020','Запросы 2021','Запросы 2022','Запросы 2023']].max(axis=1)
    df_req_min = df.loc[:,['Запросы 2020','Запросы 2021','Запросы 2022','Запросы 2023']].min(axis=1)

    df['Рейтинг по запросам 2020'] = df_req_max == df.loc[:,['Запросы 2020']].iloc[:,0]
    df['Рейтинг по запросам 2021'] = df_req_max == df.loc[:,['Запросы 2021']].iloc[:,0]
    df['Рейтинг по запросам 2022'] = df_req_max == df.loc[:,['Запросы 2022']].iloc[:,0]
    df['Рейтинг по запросам 2023'] = df_req_max == df.loc[:,['Запросы 2023']].iloc[:,0]
    df['Рейтинг по запросам, итого'] = df_req_max == df.loc[:,['Запросы, итого']].iloc[:,0]

    df['Антирейтинг по запросам 2020'] = df_req_min == df.loc[:,['Запросы 2020']].iloc[:,0]
    df['Антирейтинг по запросам 2021'] = df_req_min == df.loc[:,['Запросы 2021']].iloc[:,0]
    df['Антирейтинг по запросам 2022'] = df_req_min == df.loc[:,['Запросы 2022']].iloc[:,0]
    df['Антирейтинг по запросам 2023'] = df_req_min == df.loc[:,['Запросы 2023']].iloc[:,0]

    df['Рейтинг постоянного\nроста запросов'] = ((df['Запросы 2020'] < df['Запросы 2021']) 
        & (df['Запросы 2021'] < df['Запросы 2022'])
        & (df['Запросы 2022'] < df['Запросы 2023']))

    df['Рейтинг постоянного\nпадения запросов'] = ((df['Запросы 2020'] > df['Запросы 2021']) 
        & (df['Запросы 2021'] > df['Запросы 2022'])
        & (df['Запросы 2022'] > df['Запросы 2023']))

    df['Рейтинг по запросам ВНВН'] = ((df['Пользователи 2020'] > df['Пользователи 2021']) 
        & (df['Пользователи 2021'] < df['Пользователи 2022'])
        & (df['Пользователи 2022'] > df['Пользователи 2023']))

    df['Рейтинг по запросам НВНВ'] = ((df['Запросы 2020'] < df['Запросы 2021']) 
        & (df['Запросы 2021'] > df['Запросы 2022'])
        & (df['Запросы 2022'] < df['Запросы 2023']))

    df['Рейтинг стабильности\nчисла запросов'] = abs(df_req_min / df_req_max - 1)

    from scipy.stats.stats import pearsonr

    x = df[['Пользователи 2020','Пользователи 2021','Пользователи 2022','Пользователи 2023']]
    y = df[['Запросы 2020','Запросы 2021','Запросы 2022','Запросы 2023']]
    df = df.fillna(0)
    for row in df.index:
        df.loc[row, 'Антирейтинг схожести динамики\nпользователей и запросов'] = scipy.stats.pearsonr(x.loc[row], y.loc[row])[0]
    df['Рейтинг схожести динамики\nпользователей и запросов'] = df['Антирейтинг схожести динамики\nпользователей и запросов']


    df = row_number(df, 'Рейтинг по запросам 2020', 'Запросы 2020')
    df = row_number(df, 'Рейтинг по запросам 2021', 'Запросы 2021')
    df = row_number(df, 'Рейтинг по запросам 2022', 'Запросы 2022')
    df = row_number(df, 'Рейтинг по запросам 2023', 'Запросы 2023')
    df = row_number(df, 'Рейтинг по запросам, итого', 'Запросы, итого', filter_df=False)

    df = row_number(df, 'Антирейтинг по запросам 2020', 'Запросы 2020', desc=False)
    df = row_number(df, 'Антирейтинг по запросам 2021', 'Запросы 2021', desc=False)
    df = row_number(df, 'Антирейтинг по запросам 2022', 'Запросы 2022', desc=False)
    df = row_number(df, 'Антирейтинг по запросам 2023', 'Запросы 2023', desc=False)

    df = row_number(df, 'Рейтинг постоянного\nроста запросов', 'Запросы, итого')
    df = row_number(df, 'Рейтинг постоянного\nпадения запросов', 'Запросы, итого')
    df = row_number(df, 'Рейтинг по запросам ВНВН', 'Запросы 2023')
    df = row_number(df, 'Рейтинг по запросам НВНВ', 'Запросы 2023')
    df = row_number(df, 'Рейтинг стабильности\nчисла запросов', 'Запросы, итого', filter_df=False)

    df = row_number(df, 'Рейтинг схожести динамики\nпользователей и запросов', 'Запросы, итого', filter_df=False)
    df = row_number(df, 'Антирейтинг схожести динамики\nпользователей и запросов', 'Запросы, итого', desc=False, filter_df=False)
    print('Рейтинги построены')
    return df

def horizontal_optimization(table_to_optimize, index=45):
    new_svod = table_to_optimize
    try:
        test_svod = new_svod.drop(['InLists', 'inList, сумма'], axis=1).copy()
    except:
        test_svod = new_svod.copy()
    index_set = set(new_svod.index)
    cnt = 0
    total_sum = 9999
    while cnt < 10000 and len(index_set)> 0:
        j = index_set.pop()
        cnt += 1
        cols = [i for i in test_svod.loc[j][test_svod.loc[j] > 0].sort_values(ascending=False).index]
        for i in cols:   
            cnt_horizontal = test_svod.loc[j].drop('cnt_horizontal').replace(range(1, index+1), 1).sum()
            cnt_vertical = test_svod[i].replace(range(1, index+1), 1).sum()
            if cnt_vertical > 15 and cnt_horizontal > 1:
                test_svod[i][j] = 0
        
        cols_set = set(cols)
        cnt2 = 0
        while cols_set:
            i = cols_set.pop()
            cnt_horizontal = test_svod.loc[j].drop('cnt_horizontal').replace(range(1, index+1), 1).sum()
            cnt_vertical = test_svod[i].replace(range(1, index+1), 1).sum()
            if cnt_vertical > 15 and cnt_horizontal > 1:
                test_svod[i][j] = 0
        total_sum = test_svod.sum(axis=1).sum()
        
        #print(total_sum)
    test_svod['cnt_horizontal'] = test_svod.drop('cnt_horizontal', axis=1).replace(range(1, index+1), 1).sum(axis=1)
    test_svod = test_svod.sort_values('cnt_horizontal', ascending=False)
    print('Горизонтальная оптимизация завершена')
    return test_svod

def vertical_optimization(table_to_optimize, original_table, index=45):
    df = table_to_optimize
    new_svod = original_table
    for i in range(5):
        print('Вертикальная оптимизация шаг {} из 5'.format(i+1))
        for cat_to_optimize in df.index:
            # сейчас есть значение, превращаем в ноль
            col = df.drop('cnt_horizontal', axis=1).loc[cat_to_optimize].sort_values(ascending=False).index[0]

            # сейчас 0 возвращаем значение из бэкапа
            try:
                cat_to_reset = (
                     df
                     .loc[df.index != cat_to_optimize]
                     .loc[(new_svod.sort_index().loc[:, col] != df.sort_index().loc[:, col]) 
                          & (df[col] == 0)]
                     .loc[:,[col, 'cnt_horizontal']]
                     .sort_values([col, 'cnt_horizontal'], ascending=[False, True])
                     .index[0]
                )
                if df.loc[cat_to_reset, 'cnt_horizontal'] + 1 < df.loc[cat_to_optimize, 'cnt_horizontal']:
                    df.loc[cat_to_optimize, col] = 0
                    df.loc[cat_to_reset, col] = new_svod.loc[cat_to_reset, col]
                    df['cnt_horizontal'] = df.drop('cnt_horizontal', axis=1).replace(range(1, index+1), 1).sum(axis=1)
            except:
                #print('except vertical',i, cat_to_optimize)
                pass
    return df

def optimize_df(df_to_optimize, show_all=False, index=45, cut_level=450, calc_cut_level=False):
    original_svod = df_to_optimize.copy().iloc[:cut_level]
    print(original_svod.shape)
    original_svod_sorted = (
        original_svod
        .fillna(0)
        .set_index('Рубрика')
    ).sort_values('Рубрика')
    original_svod_sorted = original_svod.loc[[not i.isnumeric() for i in original_svod['Рубрика']]]
    
    original_svod_counted = count_ratings(original_svod_sorted).set_index('Рубрика')

    new_svod = original_svod_counted[[
     'Рейтинг по пользователям, итого',
     'Рейтинг по пользователям 2020',
     'Рейтинг по пользователям 2023',
     'Антирейтинг по пользователям 2023',
     'Антирейтинг по пользователям 2020',
     'Рейтинг постоянного\nроста пользователей',
     'Рейтинг постоянного\nпадения пользователей',
     'Рейтинг по пользователям 2021',
     'Рейтинг по пользователям 2022',
     'Антирейтинг по пользователям 2021',
     'Антирейтинг по пользователям 2022',
     'Рейтинг по пользователям ВНВН',
     'Рейтинг по пользователям НВНВ',
     'Рейтинг стабильности\nчисла пользователей',
     'Рейтинг схожести динамики\nпользователей и запросов',
     'Рейтинг по запросам, итого',
     'Рейтинг по запросам 2020',
     'Рейтинг по запросам 2023',
     'Антирейтинг по запросам 2023',
     'Антирейтинг по запросам 2020',
     'Рейтинг постоянного\nроста запросов',
     'Рейтинг постоянного\nпадения запросов',
     'Рейтинг по запросам 2021',
     'Рейтинг по запросам 2022',
     'Антирейтинг по запросам 2021',
     'Антирейтинг по запросам 2022',
     'Рейтинг по запросам ВНВН',
     'Рейтинг по запросам НВНВ',
     'Рейтинг стабильности\nчисла запросов',
     'Антирейтинг схожести динамики\nпользователей и запросов'
                ]]

    new_svod['cnt_horizontal'] = new_svod.replace(range(1, index+1), 1).sum(axis=1)
    new_svod = new_svod.sort_values('cnt_horizontal', ascending=False)

    hor_optimized_svod = horizontal_optimization(new_svod)
    vert_optimized_svod = vertical_optimization(hor_optimized_svod, new_svod)
    #vert_optimized_svod = hor_optimized_svod
    
    optimized_svod = vert_optimized_svod.reset_index()
    optimized_svod['В рейтингах, раз'] = optimized_svod['cnt_horizontal']
    #optimized_svod = optimized_svod.drop('cnt_horizontal', axis=1).sort_values('Рубрика')
    optimized_svod = optimized_svod.drop('cnt_horizontal', axis=1).sort_index()

    df = (pd.DataFrame(new_svod.value_counts('cnt_horizontal'))
     .merge(pd.DataFrame(hor_optimized_svod.value_counts('cnt_horizontal')), left_index=True, right_index=True, how='outer')
     .merge(pd.DataFrame(vert_optimized_svod.value_counts('cnt_horizontal')), left_index=True, right_index=True, how='outer')
     .fillna(0).astype(int))

    df.columns = ['Оригинальный файл', 'Горизонтальная оптимизация', 'Вертикальная оптимизация']

    display(df)
    
    if calc_cut_level:
        return optimized_svod, df
    return optimized_svod

def make_final(original_svod, optimized_svod):
    original_svod_sorted = (
        original_svod
        .fillna(0)
        .set_index('Рубрика')
    ).sort_values('Рубрика')
    original_svod_sorted = original_svod.loc[[not i.isnumeric() for i in original_svod['Рубрика']]]
    
    final_svod = original_svod_sorted.copy().reset_index()

    final_svod = final_svod[['№', 'Рубрика', 'Пользователи 2020', 'Пользователи 2021',
           'Пользователи 2022', 'Пользователи 2023', 'Пользователи, итого',
           'Пользователи, %', 'Запросы 2020', 'Запросы 2021', 'Запросы 2022',
           'Запросы 2023', 'Запросы, итого', 'Запросы, %',
           'Запросов на пользователя 2020', 'Запросов на пользователя 2021',
           'Запросов на пользователя 2022', 'Запросов на пользователя 2023',
           'Запросов на пользователя, итого']].sort_values('№')

    final_svod = final_svod.merge(optimized_svod, left_on='Рубрика', right_on='Рубрика')
    final_svod['Рейтинг суммарный'] = final_svod[optimized_svod.columns].sum(axis=1)
    final_svod['Рейтинг средний'] = final_svod[optimized_svod.columns].mean(axis=1)
    final_svod = final_svod.sort_index().sort_values('№')

    for i in optimized_svod.drop(['Рубрика', 'В рейтингах, раз'], axis=1).columns:
        final_svod = row_number(final_svod, i, i, desc=False, filter_df=False, index=15, filter_zero=True)

    final_svod = final_svod[['№', 'Рубрика', 'Пользователи 2020', 'Пользователи 2021',
           'Пользователи 2022', 'Пользователи 2023', 'Пользователи, итого',
           'Пользователи, %', 'Запросы 2020', 'Запросы 2021', 'Запросы 2022',
           'Запросы 2023', 'Запросы, итого', 'Запросы, %',
           'Запросов на пользователя 2020', 'Запросов на пользователя 2021',
           'Запросов на пользователя 2022', 'Запросов на пользователя 2023',
           'Запросов на пользователя, итого', 'Рейтинг по пользователям, итого',
           'Рейтинг по пользователям 2020', 'Рейтинг по пользователям 2021',
           'Рейтинг по пользователям 2022', 'Рейтинг по пользователям 2023',
           'Антирейтинг по пользователям 2020',
           'Антирейтинг по пользователям 2021',
           'Антирейтинг по пользователям 2022',
           'Антирейтинг по пользователям 2023',
           'Рейтинг постоянного\nроста пользователей',
           'Рейтинг постоянного\nпадения пользователей',
           'Рейтинг по пользователям ВНВН', 'Рейтинг по пользователям НВНВ',
           'Рейтинг стабильности\nчисла пользователей',
           'Рейтинг схожести динамики\nпользователей и запросов',
           'Рейтинг по запросам, итого', 'Рейтинг по запросам 2020',
           'Рейтинг по запросам 2021', 'Рейтинг по запросам 2022',
           'Рейтинг по запросам 2023', 'Антирейтинг по запросам 2020',
           'Антирейтинг по запросам 2021', 'Антирейтинг по запросам 2022',
           'Антирейтинг по запросам 2023', 'Рейтинг постоянного\nроста запросов',
           'Рейтинг постоянного\nпадения запросов', 'Рейтинг по запросам ВНВН',
           'Рейтинг по запросам НВНВ', 'Рейтинг стабильности\nчисла запросов',
           'Антирейтинг схожести динамики\nпользователей и запросов',
           'В рейтингах, раз', 'Рейтинг суммарный', 'Рейтинг средний']]
    return final_svod
    
def rename(df1_req_cnt, df1_user_cnt):
    pd_df1_req_cnt = pd.DataFrame(df1_req_cnt)
    pd_df1_user_cnt = pd.DataFrame(df1_user_cnt)
    df1_agg = pd_df1_req_cnt.merge(pd_df1_user_cnt, left_on='rubricName', right_on='rubricName')
    df1_agg = df1_agg.sort_values('date', ascending=False)
    #df1_agg['rubricName'] = df1_agg['rubricName_x']
    df1_agg = df1_agg.set_index('rubricName')[['date', 'user']].fillna(0).astype(int)
    return df1_agg

# Москва

In [ ]:
df20_raw = dd.read_csv(r"C:/Users/Mamed/Downloads/Москва_2020_969792.csv", dtype={'fl': 'object'})
df20_agg = agg_df(df20_raw, 2020)
df21_raw = dd.read_csv(r"C:/Users/Mamed/Downloads/Москва_2021_970748.csv", dtype={'fl': 'object'})
df21_agg = agg_df(df21_raw, 2021)
df22_raw = dd.read_csv(r"C:/Users/Mamed/Downloads/Москва_2022_970753.csv", dtype={'fl': 'object'})
df22_agg = agg_df(df22_raw, 2022)
df23_raw = dd.read_csv(r"C:/Users/Mamed/Downloads/Москва_2023_970754.csv", dtype={'fl': 'object'})
df23_agg = agg_df(df23_raw, 2023)

In [49]:
original_svod = pd.read_excel(r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Москва\Мастер_таблица_2ГИС_Москва_230315_1930.xlsx")

In [53]:
original_svod_filtered_optimized.to_excel('original_svod_filtered_optimized2.xlsx', index=False)

In [ ]:
Стартуем с 450 до 750 по 50 пока кол-во 5 не станет = 0
От найденного значения идем вниз по 10, пока кол-во 5 не станет > 0

In [ ]:
for cut_level_num in range(400, 900, 50):
    original_svod_filtered_optimized, df = optimize_df(original_svod, cut_level=cut_level_num, calc_cut_level=True) 
    if df.iloc[5, 2] >= 0:
        break

for cut_level_num2 in range(cut_level_num, 400, -10):
    original_svod_filtered_optimized, df = optimize_df(original_svod, cut_level=cut_level_num2, calc_cut_level=True) 
    if df.iloc[5, 2] > 0:
        break

In [90]:
original_svod_filtered_optimized = optimize_df(original_svod, cut_level=410) 

(410, 52)
Рейтинги построены
Горизонтальная оптимизация завершена
Вертикальная оптимизация шаг 1 из 5
Вертикальная оптимизация шаг 2 из 5
Вертикальная оптимизация шаг 3 из 5
Вертикальная оптимизация шаг 4 из 5
Вертикальная оптимизация шаг 5 из 5


,Оригинальный файл,Горизонтальная оптимизация,Вертикальная оптимизация
cnt_horizontal,,,
0.0,90,90,90
1.0,61,221,221
2.0,99,75,75
3.0,63,16,16
4.0,26,7,7
5.0,18,0,0
6.0,8,0,0
7.0,7,0,0
8.0,3,0,0


In [95]:
final_svod = make_final(original_svod, original_svod_filtered_optimized)

C:\Users\Mamed\AppData\Local\Temp\ipykernel_15040\1065154296.py:332: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  final_svod['Рейтинг суммарный'] = final_svod[optimized_svod.columns].sum(axis=1)
C:\Users\Mamed\AppData\Local\Temp\ipykernel_15040\1065154296.py:333: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  final_svod['Рейтинг средний'] = final_svod[optimized_svod.columns].mean(axis=1)


In [96]:
final_svod.to_excel('final_svod_master.xlsx', index=False)

# СПБ

In [ ]:
spb20_raw = dd.read_csv(r"C:\Users\Mamed\Downloads\СПБ_2020_974783.csv")
spb20_agg = agg_df(spb20_raw, 'spb_2020')

In [ ]:
spb21_raw = dd.read_csv(r"C:\Users\Mamed\Downloads\СПБ_2021_974784.csv")
spb21_agg = agg_df(spb21_raw, 'spb_2021')

# Алматы

In [ ]:
df23_raw = dd.read_csv(r"C:\Users\Mamed\Desktop\файлы_2гис_2023\Алмата_2023_974800.csv", dtype={'fl': 'object'})
df23_agg = agg_df(df23_raw, 'алмата_2023')

In [ ]:
df22_raw = dd.read_csv(r"C:\Users\Mamed\Desktop\файлы_2гис_2023\Алмата_2022_974799.csv", dtype={'fl': 'object'})
df22_agg = agg_df(df22_raw, 'алмата_2022')

In [ ]:
df21_raw = dd.read_csv(r"C:\Users\Mamed\Desktop\файлы_2гис_2023\Алмата_2021_974798.csv", dtype={'fl': 'object'})
df21_agg = agg_df(df21_raw, 'алмата_2021')

In [7]:
df20_raw = dd.read_csv(r"C:\Users\Mamed\Desktop\файлы_2гис_2023\Алмата_2020_974797.csv", dtype={'fl': 'object'})
df20_agg = agg_df(df20_raw, 'алмата_2020')

Start: 2023-03-17 03:34:44.235204
End: 2023-03-17 03:59:45.969968
Duration: 0:25:01.734764


In [7]:
df1_req_cnt_алмата_2020 = pd.read_excel('df1_req_cnt_алмата_2020.xlsx')
df1_req_cnt_алмата_2021 = pd.read_excel('df1_req_cnt_алмата_2021.xlsx')
df1_req_cnt_алмата_2022 = pd.read_excel('df1_req_cnt_алмата_2022.xlsx')
df1_req_cnt_алмата_2023 = pd.read_excel('df1_req_cnt_алмата_2023.xlsx')

df1_user_cnt_алмата_2020 = pd.read_excel('df1_user_cnt_алмата_2020.xlsx')
df1_user_cnt_алмата_2021 = pd.read_excel('df1_user_cnt_алмата_2021.xlsx')
df1_user_cnt_алмата_2022 = pd.read_excel('df1_user_cnt_алмата_2022.xlsx')
df1_user_cnt_алмата_2023 = pd.read_excel('df1_user_cnt_алмата_2023.xlsx')

df1_cnt_алмата_2020 = rename(df1_req_cnt_алмата_2020, df1_user_cnt_алмата_2020)
df1_cnt_алмата_2021 = rename(df1_req_cnt_алмата_2021, df1_user_cnt_алмата_2021)
df1_cnt_алмата_2022 = rename(df1_req_cnt_алмата_2022, df1_user_cnt_алмата_2022)
df1_cnt_алмата_2023 = rename(df1_req_cnt_алмата_2023, df1_user_cnt_алмата_2023)

df1_cnt_алмата_2020_2021 = df1_cnt_алмата_2020.merge(df1_cnt_алмата_2021, how='outer', left_index=True, right_index=True)

df1_cnt_алмата_2022_2023 = df1_cnt_алмата_2022.merge(df1_cnt_алмата_2023, how='outer', left_index=True, right_index=True)

columns1 = ['Запросы 2020','Пользователи 2020','Запросы 2021','Пользователи 2021']

columns2 = ['Запросы 2022','Пользователи 2022','Запросы 2023','Пользователи 2023']

df1_cnt_алмата_2020_2021.columns = columns1

df1_cnt_алмата_2022_2023.columns = columns2

df1_cnt_алмата = df1_cnt_алмата_2020_2021.merge(df1_cnt_алмата_2022_2023, how='outer', left_index=True, right_index=True).reset_index()

df1_cnt_алмата['Рубрика'] = df1_cnt_алмата['rubricName']

df1_cnt_алмата = df1_cnt_алмата.drop('rubricName', axis=1)

In [61]:
count_users_requests('almata', df20_raw, df21_raw, df22_raw, df23_raw)

4781734 1420348 1516714 1801050 2297946


In [8]:
almata_users_itogo = pd.read_excel('almatausers_itogo.xlsx')

almata_svod = df1_cnt_алмата.merge(almata_users_itogo, right_on='rubricName', left_on='Рубрика')

columns = ['№', 'Рубрика', 'Пользователи 2020', 'Пользователи 2021',
       'Пользователи 2022', 'Пользователи 2023', 'Пользователи, итого',
       'Пользователи, %', 'Запросы 2020', 'Запросы 2021', 'Запросы 2022',
       'Запросы 2023', 'Запросы, итого', 'Запросы, %',
       'Запросов на пользователя 2020', 'Запросов на пользователя 2021',
       'Запросов на пользователя 2022', 'Запросов на пользователя 2023',
       'Запросов на пользователя, итого']

almata_svod['Пользователи, итого'] = almata_svod['user']

for i in columns:
    if i not in almata_svod.columns:
        almata_svod[i] = ''

almata_svod = almata_svod[['№', 'Рубрика', 'Пользователи 2020', 'Пользователи 2021',
       'Пользователи 2022', 'Пользователи 2023', 'Пользователи, итого',
       'Пользователи, %', 'Запросы 2020', 'Запросы 2021', 'Запросы 2022',
       'Запросы 2023', 'Запросы, итого', 'Запросы, %',
       'Запросов на пользователя 2020', 'Запросов на пользователя 2021',
       'Запросов на пользователя 2022', 'Запросов на пользователя 2023',
       'Запросов на пользователя, итого']]

In [9]:
#almata_svod.to_excel('almata_svod.xlsx')

In [86]:
almata_svod_itogo = pd.read_excel(r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Алматы\Таблица_2ГИС_Алматы_Итого_230317_1245.xlsx")

In [87]:
almata_svod_itogo = almata_svod_itogo[almata_svod_itogo['№'] != 'Итог'].fillna(0)

In [89]:
almata_optimized = optimize_df(almata_svod_itogo, cut_level=400)

(400, 19)
Рейтинги построены
Горизонтальная оптимизация завершена
Вертикальная оптимизация шаг 1 из 5
Вертикальная оптимизация шаг 2 из 5
Вертикальная оптимизация шаг 3 из 5
Вертикальная оптимизация шаг 4 из 5
Вертикальная оптимизация шаг 5 из 5


,Оригинальный файл,Горизонтальная оптимизация,Вертикальная оптимизация
cnt_horizontal,,,
0.0,127,127,127
1.0,69,191,191
2.0,69,46,46
3.0,48,25,25
4.0,20,10,10
5.0,17,0,0
6.0,10,0,0
7.0,20,0,0
8.0,16,0,0


In [93]:
almata_final = make_final(almata_svod_itogo, almata_optimized)

C:\Users\Mamed\AppData\Local\Temp\ipykernel_15040\1065154296.py:332: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  final_svod['Рейтинг суммарный'] = final_svod[optimized_svod.columns].sum(axis=1)
C:\Users\Mamed\AppData\Local\Temp\ipykernel_15040\1065154296.py:333: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  final_svod['Рейтинг средний'] = final_svod[optimized_svod.columns].mean(axis=1)


In [94]:
almata_final.to_excel('almata_final.xlsx')

# Новосибирск

In [141]:
#2023
nov_df20_raw = dd.read_csv(r"C:\Users\Mamed\Desktop\файлы_2гис_2023\Новосибирск_2020_974796.csv", dtype={'fl': 'object'})
nov_df20_agg = agg_df(nov_df20_raw, 'новосиб_2020')

Start: 2023-03-17 04:18:04.432750
End: 2023-03-17 04:18:04.772263
Duration: 0:00:00.339513


In [142]:
#2021
nov_df21_raw = dd.read_csv(r"C:\Users\Mamed\Desktop\файлы_2гис_2023\Новосибирск_2021_974792.csv", dtype={'fl': 'object'})
nov_df21_agg = agg_df(nov_df21_raw, 'новосиб_2021')

Start: 2023-03-17 04:18:07.367428
End: 2023-03-17 04:18:07.543124
Duration: 0:00:00.175696


In [143]:
#2022
nov_df22_raw = dd.read_csv(r"C:\Users\Mamed\Desktop\файлы_2гис_2023\Новосибирск_2022_974795.csv", dtype={'fl': 'object'})
nov_df22_agg = agg_df(nov_df22_raw, 'новосиб_2022')

Start: 2023-03-17 04:18:07.859904
End: 2023-03-17 04:18:08.033286
Duration: 0:00:00.173382


In [144]:
#2020
nov_df23_raw = dd.read_csv(r"C:\Users\Mamed\Desktop\файлы_2гис_2023\Новосибирск_2023_974791.csv", dtype={'fl': 'object'})
nov_df23_agg = agg_df(nov_df23_raw, 'новосиб_2023')

Start: 2023-03-17 04:18:08.684168
End: 2023-03-17 04:18:08.855440
Duration: 0:00:00.171272


In [132]:
df1_req_cnt_новосиб_2020 = pd.read_excel('df1_req_cnt_новосиб_2020.xlsx')
df1_req_cnt_новосиб_2021 = pd.read_excel('df1_req_cnt_новосиб_2021.xlsx')
df1_req_cnt_новосиб_2022 = pd.read_excel('df1_req_cnt_новосиб_2022.xlsx')
df1_req_cnt_новосиб_2023 = pd.read_excel('df1_req_cnt_новосиб_2023.xlsx')

df1_user_cnt_новосиб_2020 = pd.read_excel('df1_user_cnt_новосиб_2020.xlsx')
df1_user_cnt_новосиб_2021 = pd.read_excel('df1_user_cnt_новосиб_2021.xlsx')
df1_user_cnt_новосиб_2022 = pd.read_excel('df1_user_cnt_новосиб_2022.xlsx')
df1_user_cnt_новосиб_2023 = pd.read_excel('df1_user_cnt_новосиб_2023.xlsx')

df1_cnt_новосиб_2020 = rename(df1_req_cnt_новосиб_2020, df1_user_cnt_новосиб_2020)
df1_cnt_новосиб_2021 = rename(df1_req_cnt_новосиб_2021, df1_user_cnt_новосиб_2021)
df1_cnt_новосиб_2022 = rename(df1_req_cnt_новосиб_2022, df1_user_cnt_новосиб_2022)
df1_cnt_новосиб_2023 = rename(df1_req_cnt_новосиб_2023, df1_user_cnt_новосиб_2023)

df1_cnt_новосиб_2020_2021 = df1_cnt_новосиб_2020.merge(df1_cnt_новосиб_2021, how='outer', left_index=True, right_index=True)

df1_cnt_новосиб_2022_2023 = df1_cnt_новосиб_2022.merge(df1_cnt_новосиб_2023, how='outer', left_index=True, right_index=True)

columns1 = ['Запросы 2020','Пользователи 2020','Запросы 2021','Пользователи 2021']

columns2 = ['Запросы 2022','Пользователи 2022','Запросы 2023','Пользователи 2023']

df1_cnt_новосиб_2020_2021.columns = columns1

df1_cnt_новосиб_2022_2023.columns = columns2

df1_cnt_новосиб = df1_cnt_новосиб_2020_2021.merge(df1_cnt_новосиб_2022_2023, how='outer', left_index=True, right_index=True).reset_index()

df1_cnt_новосиб['Рубрика'] = df1_cnt_новосиб['rubricName']

df1_cnt_новосиб = df1_cnt_новосиб.drop('rubricName', axis=1)

In [145]:
count_users_requests('nov', nov_df20_raw, nov_df21_raw, nov_df22_raw, nov_df23_raw)

2785152 1212936 1067135 1158251 1084689


##### исправление ошибки с перепутанными датами
df1_cnt_новосиб['Запросы 2020'], df1_cnt_новосиб['Запросы 2023'] = df1_cnt_новосиб['Запросы 2023'], df1_cnt_новосиб['Запросы 2020']

df1_cnt_новосиб['Пользователи 2020'], df1_cnt_новосиб['Пользователи 2023'] = df1_cnt_новосиб['Пользователи 2023'], df1_cnt_новосиб['Пользователи 2020']

In [155]:
nov_users_itogo = pd.read_excel('novusers_itogo.xlsx')

In [165]:
novosib_svod = df1_cnt_новосиб.merge(nov_users_itogo, right_on='rubricName', left_on='Рубрика')

In [169]:
columns = ['№', 'Рубрика', 'Пользователи 2020', 'Пользователи 2021',
       'Пользователи 2022', 'Пользователи 2023', 'Пользователи, итого',
       'Пользователи, %', 'Запросы 2020', 'Запросы 2021', 'Запросы 2022',
       'Запросы 2023', 'Запросы, итого', 'Запросы, %',
       'Запросов на пользователя 2020', 'Запросов на пользователя 2021',
       'Запросов на пользователя 2022', 'Запросов на пользователя 2023',
       'Запросов на пользователя, итого']

In [170]:
novosib_svod['Пользователи, итого'] = novosib_svod['user']

In [177]:
for i in columns:
    if i not in novosib_svod.columns:
        novosib_svod[i] = ''

In [179]:
novosib_svod = novosib_svod[['№', 'Рубрика', 'Пользователи 2020', 'Пользователи 2021',
       'Пользователи 2022', 'Пользователи 2023', 'Пользователи, итого',
       'Пользователи, %', 'Запросы 2020', 'Запросы 2021', 'Запросы 2022',
       'Запросы 2023', 'Запросы, итого', 'Запросы, %',
       'Запросов на пользователя 2020', 'Запросов на пользователя 2021',
       'Запросов на пользователя 2022', 'Запросов на пользователя 2023',
       'Запросов на пользователя, итого']]

In [182]:
#novosib_svod.to_excel('novosib_svod.xlsx')

In [80]:
novosib_svod_itogo = pd.read_excel(r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Новосибирск\Таблица_2ГИС_Новосибирск_Итого_230317_1245.xlsx")

In [81]:
novosib_svod_itogo = novosib_svod_itogo[novosib_svod_itogo['№'] != 'Итог'].fillna(0)

In [83]:
novosib_filtered_optimized = optimize_df(novosib_svod_itogo_filtered, cut_level=400) 

(400, 19)
Рейтинги построены
Горизонтальная оптимизация завершена
Вертикальная оптимизация шаг 1 из 5
Вертикальная оптимизация шаг 2 из 5
Вертикальная оптимизация шаг 3 из 5
Вертикальная оптимизация шаг 4 из 5
Вертикальная оптимизация шаг 5 из 5


,Оригинальный файл,Горизонтальная оптимизация,Вертикальная оптимизация
cnt_horizontal,,,
0.0,10,10,10
1.0,38,309,309
2.0,139,57,57
3.0,105,23,23
4.0,44,1,1
5.0,16,0,0
6.0,8,0,0
7.0,14,0,0
8.0,6,0,0


In [84]:
novosib_filtered_optimized_final = make_final(novosib_svod_itogo, novosib_filtered_optimized)

C:\Users\Mamed\AppData\Local\Temp\ipykernel_15040\1065154296.py:332: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  final_svod['Рейтинг суммарный'] = final_svod[optimized_svod.columns].sum(axis=1)
C:\Users\Mamed\AppData\Local\Temp\ipykernel_15040\1065154296.py:333: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  final_svod['Рейтинг средний'] = final_svod[optimized_svod.columns].mean(axis=1)


In [91]:
novosib_filtered_optimized_final.to_excel('novosib_filtered_optimized_final.xlsx')